In [80]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import seawater as sea

# Open the NetCDF file
ds_temp = nc.Dataset('IonianTemperature.nc', 'r')  # Replace 'path_to_your_file.nc' with the actual file path


# Validate This:
# P = P_0 + ρgh // pressure, P_0 in hPa
# def findPressure(depth):
#     P_0 = 1013.25
#     density = 1025 # can slightly vary (kg/m^3)
#     g = 9.81 # m/s^2
#     h = depth # m
#     return P_0 + density*g*h

 
temp = ds_temp.variables['thetao']
depth = ds_temp.variables['depth']
lat = ds_temp.variables['latitude']
long = ds_temp.variables['longitude']
time = ds_temp.variables['time']

depths = depth[:]
times = time[:]


latitude,longitude = ds_temp.variables['latitude'], ds_temp.variables['longitude']

# extract lat/lon values (in degrees) to numpy arrays
latvals = latitude[:]; lonvals = longitude[:]

def get_closest_ij(lats, lons, latpt, lonpt):
    # Create 2D grid arrays for lats and lons
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    # Calculate squared distance from every grid point to the given point
    # Notice that we are using lat_grid and lon_grid which are 2D arrays
    dist_sq = (lat_grid - latpt)**2 + (lon_grid - lonpt)**2
    
    # Find the 1D index of the minimum squared distance
    minindex_flattened = dist_sq.argmin()
    
    # Convert the 1D index back to 2D indices for lat and lon arrays
    ij_min = np.unravel_index(minindex_flattened, dist_sq.shape)
    
    return ij_min

iy_min, ix_min = get_closest_ij(latvals, lonvals, 50., -140)
print(iy_min)
print(ix_min)


96
0


In [74]:
import numpy as np

def findex(masked_array, target_value):
    """
    Finds the index of the closest value to the target_value in a masked array.

    Parameters:
    - masked_array: np.ma.MaskedArray, the array to search within.
    - target_value: float or int, the value to find the closest to in masked_array.

    Returns:
    - tuple: The index of the closest value in the masked_array.
    """
    # Ensure the input is a masked array
    if not isinstance(masked_array, np.ma.MaskedArray):
        raise ValueError("Input must be a masked array.")

    # Compute the absolute difference with the target value, ignoring masked values
    absolute_differences = np.ma.abs(masked_array - target_value)

    # Find the index of the minimum difference (closest value)
    closest_index = np.ma.argmin(absolute_differences)

    # np.ma.argmin returns a flattened index if the array is multidimensional,
    # so you might need to convert it to a multidimensional index depending on your needs
    if masked_array.ndim > 1:
        closest_index = np.unravel_index(closest_index, masked_array.shape)

    return closest_index


In [111]:
ds_temp = nc.Dataset('IonianSalinity.nc', 'r')
ds_horiz= nc.Dataset('IonianHorizontal.nc', 'r')
ds_vertical= nc.Dataset('IonianVertical.nc', 'r')


east_vel = ds_horiz['uo']
north_vel = ds_vertical['wo']
sal = ds_temp['so']


''' TEST '''
# time, depth
closestTime = findex(times,0)
closestDepth = findex(depths,100)

# choose your own lat, long
iy_min, ix_min = get_closest_ij(latvals, lonvals, 50., -140)

# use units of original array, thus use
#print('%7.4f %s' % (east_vel[closestTime,closestDepth,iy_min,ix_min], east_vel.units))

''' TEST '''
# for any proper indices:
# (time,depth,latitude,longitude)

'''1st''' 

'''find given density at depth using equation''' 
def findPressure(depthUsed):
    return 1013.25 + (1025*9.81*depthUsed)/(10**4)

def findDensity(lat,long,depthUsed):
    pressure = findPressure(depthUsed)

    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)

    # sal at lat long depth
    salUsed = sal[0,depthFound,y_min,x_min]

    # temp at lat long depth
    tempUsed = temp[0,depthFound,y_min,x_min]

    return sea.dens(salUsed,tempUsed,pressure)

# sometimes calculation is -- idk why
print(findDensity(25,25,0))
'''2nd'''
'''find given velocity at depth using data'''
def findVelocity(lat,long,depthUsed):
    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)
    horiz_v = east_vel[0,depthFound,y_min,x_min]
    vert_v = north_vel[0,depthFound,y_min,x_min]
    return [horiz_v,vert_v]

vels = (findVelocity(25,25,0))
print(vels[0],vels[1])

1030.1926513733088
0.14336081 6.8225677e-07


In [132]:
import random
'''3rd'''
'''Factor typical rising speed (mechanical,ballasts?), then slow with provisions,'''

# modify previous functions to include time component (which only matters if a day elapses)
def findDensity(lat,long,depthUsed,timeUsed):
    pressure = findPressure(depthUsed)

    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)

    # sal at lat long depth
    salUsed = sal[0+timeUsed,depthFound,y_min,x_min]

    # temp at lat long depth
    tempUsed = temp[0+timeUsed,depthFound,y_min,x_min]

    return sea.dens(salUsed,tempUsed,pressure)

# from sea currents
def findVelocity(lat,long,depthUsed,timeUsed):
    y_min, x_min = get_closest_ij(latvals, lonvals, lat, long)
    depthFound = findex(depths,depthUsed)
    horiz_v = east_vel[0+timeUsed,depthFound,y_min,x_min]
    vert_v = north_vel[0+timeUsed,depthFound,y_min,x_min]
    return [horiz_v,vert_v]

# calculate the Δv using Δv = a * Δt
# and we are using intervals of 1 second
def updateVelocity(vel_x, vel_y, sea_vel_x, sea_vel_y ):
    A = 3.6*2.4 # h * w in m
    CD = .25 # values between .1 and .4 for a sphere, picked avg
    
    x_A = CD*A*1025*(1/2)*(sea_vel_x)**2
    y_A = CD*A*1025*(1/2)*(sea_vel_y)**2

    vel_x -= x_A
    vel_y -= y_A
    return [vel_x,vel_y]

# arbitrary points (if time)
# determined vel
'''Model Path using constant (then decaying speeds)'''
'''Factor typical rising speed (mechanical,ballasts?), then slow with provisions,'''
def simSub(v_x,v_y,v_z, x,y,z, riseSpeed):

    # arbitrary amounts yet again
    # while ( oxygenated & powered )
    timer = 0


    
    curr_x = x
    curr_y = y
    curr_z = z

    curr_v_x = v_x
    curr_v_y = v_y
    curr_v_z = v_z

    while  (curr_z > 10):
        
        # check deadpoint

        # find accel
        sea_vel_x = findVelocity[curr_x,curr_y,curr_z,timer][0]
        sea_vel_y = findVelocity[curr_x,curr_y,curr_z,timer][1]

        curr_v_x = updateVelocity(curr_v_x,curr_v_y,curr_x,curr_y)[0]
        curr_v_y = updateVelocity(curr_v_x,curr_v_y,curr_x,curr_y)[1]

        # iterate on vel # 111,111 metres is a lat or long
        curr_x += curr_v_x/111111
        curr_y += curr_v_y/111111

        # upward movement !
        curr_z -= riseSpeed

        # repeat, think of cases???
        time += 1
        


0.494025

In [ ]:
min(depths)
max(depths)

def helpSeed(low,high,arr):
    val = random.uniform(low,high)
    index = findex(val,arr)
    
    return index

def seedSubVals():

# random.uniform(1, 10)
# then findex that

# randLat = 15.166 to 23.1666
# randLong = 36.41668 to 40.5
# randDepth = (.49) 4.9 e-01 to 1245
# rand_v_X = up to 3 kn (2-3)
# rand_v_Y = turning case, (irrelevant?) (.5 to 1)??? ***
# rand_v_Z =                                          ***

# simSub(v_x,v_y,v_z, x,y,z, riseSpeed)



# 4th
# Model Path using constant (then decaying speeds)
# Path: { Buoyancy, Drag, || Vectors}

# 5th, Simulations
# Error
# Carribbean Case Study?
# Cases
# Error Definition and Generalization
# Finally Visualization

# Close the dataset
# ds.close()
